# Lottery gamifier

### Methodology
* Choose random retailers meeting certain criteria weighted based on the purchasing ability of retailers (previous day order is considered as purchasing ability)
* Choose the number of random retailers based on the weightage of number of retailers under each AM & retail group
* Choose the number of retailers (one fourth) of total retailers in a day

### Steps

1. Import packages
2. Import data
3. Remove null
4. Occurence of Retailer
5. Weightage for AM & Retail group
6. Split data based on AM's & Retail group
7. Function to generate random Rtr's by weight of AM Retail group
8. Move last day from Results to History
9. Read & join csv in History
10. Main logic with Voila for Web app

### Import packages

In [1]:
from ckpackages import azsql #customer package
import numpy as np
import pandas as pd
import random
import datetime as dt

### Import data

In [2]:
Query = 'select * from v_PC_MeeraRetailerCampaign' 
#Transactions for Meera Powder
data = azsql.callstatement(Query)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1978 entries, 0 to 1977
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   RtrCode         1978 non-null   object        
 1   RtrName         1978 non-null   object        
 2   RtrGroupCde     1962 non-null   object        
 3   DistName        1978 non-null   object        
 4   BillNo          1978 non-null   object        
 5   InvDate         1978 non-null   datetime64[ns]
 6   Brand           1978 non-null   object        
 7   PrdName         1978 non-null   object        
 8   StateName       0 non-null      object        
 9   ZM              1978 non-null   object        
 10  AMName          1978 non-null   object        
 11  TSOName         1978 non-null   object        
 12  SMSysName       1978 non-null   object        
 13  TotalQty        1978 non-null   float64       
 14  PolypouchCount  1978 non-null   float64       
 15  FinY

In [4]:
data.head()

,RtrCode,RtrName,RtrGroupCde,DistName,BillNo,InvDate,Brand,PrdName,StateName,ZM,AMName,TSOName,SMSysName,TotalQty,PolypouchCount,FinYr,JC,JCWk
0,S100000031550047076,BHAVANI PROVISION STORE,PC-KIRANA/GENERAL STORES,PONNI & CO,CKBIL2008550,2020-10-26,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,150.0,1.0,2020-21,JC08,WK02
1,GT0100020075280002007528-SLM0031584524384,Mathura super market,PC-Super Market (MT in GT & Departmental Store),PONNI & CO,CKBIL2008151,2020-10-21,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,450.0,3.0,2020-21,JC08,WK01
2,TNSSR009-SLM0011551259675,K.N. Brothers,PC-WHOLESALE,PONNI & CO,CKBIL2008261,2020-10-22,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,1350.0,9.0,2020-21,JC08,WK02
3,S100000221550322447,Lakshmi STORE,PC-KIRANA/GENERAL STORES,HARISH KANNA AGENCY,CKBIL2012296,2020-10-24,Meera Shampoo,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,KUMAR K,150.0,1.0,2020-21,JC08,WK02
4,S100000021551525328,VIJALAKSHMI STORE,PC-KIRANA/GENERAL STORES,PONNI & CO,CKBIL2008332,2020-10-23,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,150.0,1.0,2020-21,JC08,WK02


In [5]:
data['weekday'] = data['InvDate'].dt.dayofweek

In [6]:
currentWeekDay = dt.datetime.now().weekday()

In [7]:
# Data only for Latest date
# On monday, get Sat & Sun data, for other days, get previous day data
if currentWeekDay==0:
    cutoff_date = data['InvDate'].max() - pd.Timedelta(days=1)
    max_date = data['InvDate'].max()
    
    data = data[(data['InvDate'] >= cutoff_date)]
    data = data[(data['InvDate'] <= max_date)]
else:
    data = data[data['InvDate'] == data['InvDate'].max()]

In [8]:
data.head(2)

,RtrCode,RtrName,RtrGroupCde,DistName,BillNo,InvDate,Brand,PrdName,StateName,ZM,AMName,TSOName,SMSysName,TotalQty,PolypouchCount,FinYr,JC,JCWk,weekday
0,S100000031550047076,BHAVANI PROVISION STORE,PC-KIRANA/GENERAL STORES,PONNI & CO,CKBIL2008550,2020-10-26,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,150.0,1.0,2020-21,JC08,WK02,0
6,S100000031551860860,S GANAPATHY STORE,PC-KIRANA/GENERAL STORES,PONNI & CO,CKBIL2008544,2020-10-26,Meera Powder & Paste,MEERA HRB PWD 5.6 GM 1350 PCS DIWALI OFF,None,C PERUMAL,BALAJI S,ASHOK KUMAR R,GOWTHAM B,150.0,1.0,2020-21,JC08,WK02,0


In [9]:
data.InvDate.unique()

array(['2020-10-26T00:00:00.000000000'], dtype='datetime64[ns]')

### Remove null

In [10]:
# Retailers without AM
print(data['AMName'].isnull().sum())
data[data['AMName'].isnull()]

0


,RtrCode,RtrName,RtrGroupCde,DistName,BillNo,InvDate,Brand,PrdName,StateName,ZM,AMName,TSOName,SMSysName,TotalQty,PolypouchCount,FinYr,JC,JCWk,weekday


In [12]:
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

In [13]:
data['RtrGroupCde1'] = data['RtrGroupCde'].apply(lambda x: 'PC-WHOLESALE' if x=='PC-WHOLESALE' else 'PC-RETAIL')

In [14]:
# Filter wholesale retailers
#data = data[data['RtrGroupCde1'] == 'PC-WHOLESALE']

### Occurence of Retailer

In [15]:
# List of retailer
rt = data['RtrCode'].tolist()
rt.sort()
rtr = pd.Series(rt).value_counts()
rtr_list = pd.DataFrame(rtr,columns = ['retailer'] )
rtr_list['retailer']

GT0100020001360002000136-SLM0011579689437    2
S100000451552283974                          2
TNPAA005-SLM0031552456412                    2
GT0100020001140002000114-SLM0021561025513    2
TNSPA013-SLM0011552293788                    2
                                            ..
GT0100020001430002000143-SLM0011561364574    1
S100000381550912490                          1
TNVIA013-SLM0011552115388                    1
GT0100020000770002000077-SLM0011556092297    1
GT0100020001360002000136-SLM0011557303441    1
Name: retailer, Length: 89, dtype: int64

In [33]:
# Maximum occurence of retailer
rtr_list['retailer'].max()

5

In [16]:
rtr_master = data[['RtrCode','RtrName','SMSysName','DistName','RtrGroupCde1','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()

In [17]:
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

In [18]:
# Choose number of retailers to choose
pick = 75

### Weightage for AM

In [114]:
# Percentage of Retailers across AM-Wise
#TotalRtrCount = data['RtrCode'].count()
#AM_RTR_WGT = data.groupby('AMName')['RtrCode'].count().reset_index()
#AM_RTR_WGT['WGT%'] = np.ceil((AM_RTR_WGT['RtrCode']/TotalRtrCount) * pick)

#AM_RTR_WGT

In [19]:
# Percentage of Retailers across AM-Wise & Retailer group wise
TotalRtrCount = data['RtrCode'].count()
AM_RTR_WGT = data.groupby(['AMName','RtrGroupCde1'])['RtrCode'].count().reset_index()
AM_RTR_WGT['WGT%'] = np.ceil((AM_RTR_WGT['RtrCode']/TotalRtrCount) * pick)

AM_RTR_WGT

,AMName,RtrGroupCde1,RtrCode,WGT%
0,BALAJI S,PC-WHOLESALE,23,18.0
1,SAFFIULLAH R,PC-WHOLESALE,38,30.0
2,SYED IBRAHIM,PC-WHOLESALE,30,24.0
3,V BHARATHI,PC-WHOLESALE,5,4.0


In [20]:
TotalRtrCount

96

### Split dataframe on AM's

In [21]:
# Select required rows
data1 = data[['ZM','AMName','SMSysName','RtrGroupCde1','RtrCode','RtrName','TotalQty','PolypouchCount']]

In [22]:
data2 = [rows for _, rows in data1.groupby(['AMName','RtrGroupCde1'])]

In [44]:
data2[0]
data2[1]
data2[2]
data2[3]

,ZM,AMName,SMSysName,RtrGroupCde1,RtrCode,RtrName,TotalQty,PolypouchCount
840,C PERUMAL,SAFFIULLAH R,BHARATH,PC-WHOLESALE,S100000681551162809,RADHA STORE,375.0,2.5
890,C PERUMAL,SAFFIULLAH R,MANIKANDAN S,PC-WHOLESALE,S100000601551766148,MUTHU STORE,2700.0,18.0
965,C PERUMAL,SAFFIULLAH R,JEEVAGAN,PC-WHOLESALE,S100000661550907010,SAROJA STORES,13500.0,90.0
968,C PERUMAL,SAFFIULLAH R,SIVARAM,PC-WHOLESALE,GT0100020001200002000120-SLM0021564152210,SAKTHIMURUGAN STORE,13500.0,90.0
977,C PERUMAL,SAFFIULLAH R,SUBA SANJAY,PC-WHOLESALE,S100000551557830324,jai rajputana marketing,1350.0,9.0
1037,C PERUMAL,SAFFIULLAH R,ASHOK VILSAN,PC-WHOLESALE,TNRST003-SLM0011552631513,EASWARAN SETTIYAR CENTRAL MARKET,150.0,1.0
1049,C PERUMAL,SAFFIULLAH R,KUPPUSAMY,PC-WHOLESALE,GT0100020001120002000112-SLM0021557483209,RAHAMATH STORE,1350.0,9.0


## Function to generate random Rtr's

### Case: With repetitions (allowing 1st two instances of duplicates)

In [23]:
def randomRtrAmWise(i):
    # Retailer name
    a = data2[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data2[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['WGT%'][i])
#    random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R
    
def randompick(AM_RTR_WGT):
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_WGT.index.tolist())):
        df = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)
    return df_RtrRP

def randompickAM():
    result = randompick(AM_RTR_WGT)
    result = result.merge(rtr_master,on='RtrCode')
    result = result.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates
    index = result.index
    number_of_rows = len(index)
    print('Count of picked retailers: ',number_of_rows)
    #result.to_csv('results').csv',index=False)
    return result

In [46]:
randompickAM()

Count of picked retailers:  53


,RtrCode,AMName,RtrName,SMSysName,DistName,RtrGroupCde1,RtrGroupCde,InvDate,PolypouchCount
0,GT0100020000770002000077-SLM0011556090444,BALAJI S,PARSV PADMAVATI ENTERPRISES,LOGU,ARIHANT DISTRIBUTORS,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
1,GT0100020000770002000077-SLM0011556093164,BALAJI S,V. N. ARUMUGAM &CO,LOGU,ARIHANT DISTRIBUTORS,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
2,S100000011550661438,BALAJI S,SANJANA STORE,THIRUMOORTHI V,PONNI & CO,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
3,S100000161550640721,BALAJI S,SRI KAMATCHI STORE,TAMILSELVAN,PONNI & CO,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
4,S100000161563351070,BALAJI S,SRI THIRUPTHI TRADERS,TAMILSELVAN,PONNI & CO,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
5,S100000161563351070,BALAJI S,SRI THIRUPTHI TRADERS,TAMILSELVAN,PONNI & CO,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
6,S100000401550574992,BALAJI S,Perumal store,KRISHNA MOORTHY,TP DISTRIBUTION,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
7,S100000441550736349,BALAJI S,NANDHINI STORE,JAYENDRAKUMAR M,EVERGREEN ENTERPRISES,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,45
8,S100000451551082187,BALAJI S,Sri tharani store,JEEVAJOTHI C,EVERGREEN ENTERPRISES,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9
9,S100000451551083515,BALAJI S,Sri jai narayana stores,JEEVAJOTHI C,EVERGREEN ENTERPRISES,PC-WHOLESALE,PC-WHOLESALE,2020-10-26,9


## Other functions

In [ ]:
# Numpy & Pandas for calcs & dataframe
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

# Random pick
import random

# Azure data & credentials
from ckpackages import azsql

# Move files to folder
import shutil
import os,time
import datetime
import datetime as dt

# Merge csv into one
from glob import glob

# IPY - VOILA 
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import widgets,Button, Output, VBox,interactive,interact,IntProgress
import time

### Move last day from Results to History

In [ ]:
## Move last day results to history
path = os.getcwd()

now = dt.datetime.now()
ago = now-dt.timedelta(hours=24)
strftime = "%H:%M %m/%d/%Y"
created = os.path.join(path + '\Results_csv')
dest = os.path.join(path + '\History_csv')

# Detect files from one folder created or modified in the past 24 hours
for root, dirs,files in os.walk(created):  
    for fname in files:
        path = os.path.join(root, fname)
        st = os.stat(path)    
        mtime = dt.datetime.fromtimestamp(st.st_mtime)
        if mtime > ago:
            print("True:  ", fname, " at ", mtime.strftime("%H:%M %m/%d/%Y"))
            shutil.move(path, dest)
            # this is actual move

### Read & join csv in History

In [ ]:
## Read & join csv in same folder
a = os.getcwd()
path = a + '\\History_csv'
all_files = glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df_concat = pd.concat(li, axis=0, ignore_index=True)

# Convert InvDate to Date format
df_concat['InvDate'] = pd.to_datetime(df_concat['InvDate'])

## Filter data by instances occuring in last week
days=7 #Last 6 days from current date
cutoff_date = df_concat['InvDate'].max() - pd.Timedelta(days=days)
max_date = df_concat['InvDate'].max()
#print(cutoff_date)

df1_concat = df_concat[(df_concat['InvDate'] >= cutoff_date)]
df1_concat = df_concat[(df_concat['InvDate'] < max_date)]

# List of last 6 days unique retailers
rtr_last6days = df1_concat[['RtrCode','RtrName']].drop_duplicates()
rtrlist_last6days = rtr_last6days['RtrCode'].tolist()
rtrnames_last6days = rtr_last6days['RtrName'].tolist()

# Filter retailer occuring in last 6 days (to be included below)
#data_filtered = data[~data.RtrCode.isin(rtr_last6days)]

### Main logic with Voila for Web app

In [ ]:
## Import data
Query = 'select * from v_PC_MeeraRetailerCampaign' 
data = azsql.callstatement(Query)

# Add weekday & current weekday
data['weekday'] = data['InvDate'].dt.dayofweek
currentWeekDay = dt.datetime.now().weekday()

# Data only for Latest date
# On monday, get Sat & Sun data, for other days, get previous day data
if currentWeekDay==0:
    cutoff_date = data['InvDate'].max() - pd.Timedelta(days=1)
    max_date = data['InvDate'].max()
    
    data = data[(data['InvDate'] >= cutoff_date)]
    data = data[(data['InvDate'] <= max_date)]
else:
    data = data[data['InvDate'] == data['InvDate'].max()]

## Remove null - Retailers without AM
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

# Categories retail group as Wholesale & Retail
data['RtrGroupCde1'] = data['RtrGroupCde'].apply(lambda x: 'PC-WHOLESALE' if x=='PC-WHOLESALE' else 'PC-RETAIL')

# Filter retailer occuring in last 6 days
data = data[~data.RtrCode.isin(rtr_last6days)]

InvDate =  data['InvDate'].iloc[0]
InvDate = str(InvDate)[0:10]

maxdate= str(data['InvDate'].max())[0:10]
print('\nDate: ',maxdate)

TotalRtrCount = data['RtrCode'].count()
print('Unique retailers:', TotalRtrCount)

## Retailer master
rtr_master = data[['RtrCode','RtrName','SMSysName','DistName','RtrGroupCde1','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

AM_RTR_in = data.groupby(['AMName','RtrGroupCde1'])['RtrCode'].count().reset_index()

# Create text widget for output
output_slider_variable = widgets.Text()


In [ ]:
# Define function to bind value of the input to the output variable 
def f(Lots):
    output_slider_variable.value = str(Lots)

print('\nChoose number of Lots')
# Create input slider with default value = 10  
interact(f, Lots=(10, 100, 1))

def picks(output_slider_variable,data):
    # Create and output new int variable with value of slider
    new_variable = int(output_slider_variable.value)

    ## Choose number of retailers to choose
    ## Weightage for AM
    # Percentage of Retailers across AM-Wise        
    TotalRtrCount = data['RtrCode'].count()
    AM_RTR_WGT = data.groupby(['AMName','RtrGroupCde1'])['RtrCode'].count().reset_index()
    AM_RTR_WGT['LotsPerAM'] = np.ceil((AM_RTR_WGT['RtrCode']/TotalRtrCount) * new_variable)


    ## Split dataframe on AM's
    # Select required rows
    data1 = data[['ZM','AMName','RtrGroupCde1','RtrCode','RtrName','TotalQty','PolypouchCount']]
    data2 = [rows for _, rows in data1.groupby(['AMName','RtrGroupCde1'])]
    return data2,AM_RTR_WGT,new_variable
    
## Function to generate random Rtr's with AM weightage
def randomRtrAmWise(i):
    data3,AM_RTR_WGT,lots = picks(output_slider_variable,data)
    # Retailer name
    a = data3[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data3[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['LotsPerAM'][i])
    #random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R,AM_RTR_WGT,lots
    
def randompick(rtr_master,AM_RTR_in):
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_in.index.tolist())):
        df,AM_RTR_WGT,lots = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)

    df_RtrRP = df_RtrRP.merge(rtr_master,on='RtrCode')
    df_RtrRP = df_RtrRP.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates

    index = df_RtrRP.reset_index().index
    number_of_rows = len(index)
    return df_RtrRP,lots,number_of_rows,AM_RTR_WGT

def randompickAM(rtr_master,AM_RTR_in):
    result,lots,number_of_rows,AM_RTR_WGT = randompick(rtr_master,AM_RTR_in)
    while (number_of_rows!=lots):
        if (number_of_rows==lots):
            display(result)
            print('Ok')
        else:
            result,lots,number_of_rows,AM_RTR_WGT = randompickAM(rtr_master,AM_RTR_in)
            break
    return result,lots,number_of_rows,AM_RTR_WGT


In [ ]:
print('\n')
# Button pick
button = Button(description='Pick')

out = Output()


def on_click(_):
    with out:
        clear_output(True)
        dr,lots,number_of_rows,AM_RTR_WGT = randompickAM(rtr_master,AM_RTR_in)
        
        max_count = 100

        f = IntProgress(min=0, max=max_count) # instantiate the bar
        display(f) # display the bar

        count = 0
        while count <= max_count:
            f.value += 1 # signal to increment the progress bar
            time.sleep(.1)
            count += 1
                
        print('Count of lots: ',lots)
        print('Count of picked retailers: ',number_of_rows)
        print('\n Split of Retailers chosen acorss AMs')
        display(AM_RTR_WGT)
        
        print('\nThe selected retailers on date', InvDate)
        display(dr)
        
        # Save results
        root = 'Results_csv' 
        sheet = 'results_' + InvDate + '.csv' 
        file_name = os.path.join(root,sheet)
        print(file_name)
        dr.to_csv(file_name,index=False)

button.on_click(on_click)

VBox([button])

In [ ]:
VBox([out])